# Hướng dẫn Finetune UniVL trên Dataset MSRVTT cho Video Captioning

## 🚀 Training trên Kaggle

Notebook này được thiết kế để chạy trên **Kaggle** với GPU miễn phí.

### Yêu cầu Kaggle:
- **GPU**: Chọn GPU T4 x2 hoặc P100 trong Settings
- **Internet**: Bật Internet ON để download data
- **Persistence**: Bật để giữ session

---

## Tổng quan về UniVL

**UniVL** (Unified Video and Language Pre-Training Model) là một mô hình pre-trained cho cả hiểu và sinh ngôn ngữ từ video. Mô hình được thiết kế với:

- **4 modules chính:**
  1. **Text Encoder (BERT)**: Mã hóa văn bản
  2. **Visual Encoder**: Mã hóa đặc trưng video
  3. **Cross Encoder**: Kết hợp thông tin video-text
  4. **Decoder**: Sinh caption từ video

- **5 objectives training:**
  - Masked Language Modeling (MLM)
  - Masked Frame Modeling (MFM)
  - Video-Text Matching (VTM)
  - Video-Text Alignment (VTA)
  - Language Modeling (LM)

## Về Dataset MSRVTT

**MSRVTT** (Microsoft Research Video to Text) bao gồm:
- 10,000 video clips
- Mỗi video có ~20 câu mô tả (captions)
- Train: 6,513 videos (video0-video6512)
- Val: 497 videos (video6513-video7009)
- Test: 2,990 videos (video7010-video9999)

---
## Bước 0: Clone Source Code UniVL từ GitHub

### 0.1 Clone repository và cài đặt dependencies

In [1]:
# Clone UniVL repository
!git clone https://github.com/phihungcr1701/UniVL-Video-captioning.git
%cd UniVL-Video-captioning

# Kiểm tra cấu trúc thư mục
!ls -la

Cloning into 'UniVL-Video-captioning'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 64 (delta 16), reused 62 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (64/64), 203.00 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (16/16), done.
/kaggle/working/UniVL-Video-captioning
total 660
drwxr-xr-x 7 root root   4096 Dec 13 13:52 .
drwxr-xr-x 4 root root   4096 Dec 13 13:52 ..
drwxr-xr-x 3 root root   4096 Dec 13 13:52 assets
drwxr-xr-x 2 root root   4096 Dec 13 13:52 dataloaders
drwxr-xr-x 8 root root   4096 Dec 13 13:52 .git
-rw-r--r-- 1 root root     10 Dec 13 13:52 .gitignore
-rw-r--r-- 1 root root   1066 Dec 13 13:52 LICENSE
-rw-r--r-- 1 root root  19914 Dec 13 13:52 main_pretrain.py
-rw-r--r-- 1 root root  36932 Dec 13 13:52 main_task_caption.py
-rw-r--r-- 1 root root  24353 Dec 13 13:52 main_task_retrieval.py
-rw-r--r-- 1 root root    796 Dec 13 13:52 metrics.py
drwxr-x

### 0.2 Kiểm tra môi trường Kaggle

In [2]:
import sys
import torch

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Make sure to enable GPU in Kaggle Settings.")

# Kiểm tra thư mục làm việc
!pwd

Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
PyTorch version: 2.6.0+cu124
CUDA available: False
⚠️ WARNING: No GPU detected! Make sure to enable GPU in Kaggle Settings.
/kaggle/working/UniVL-Video-captioning


### 0.3 Cài đặt dependencies

Cài đặt các thư viện cần thiết cho UniVL:

In [3]:
!pip install tqdm boto3 requests pandas -q

In [4]:
# ✅ Cài đặt pycocoevalcap để tính evaluation metrics (thay thế nlg-eval)
print("\n📦 Đang cài đặt pycocoevalcap cho evaluation metrics...")
!pip install git+https://github.com/salaniz/pycocoevalcap.git -q

print("\n✓ Đã cài đặt xong!")
print("\n📝 Các thư viện đã cài:")
print("- NumPy < 1.24 (tương thích với np.long)")
print("- tqdm, boto3, requests, pandas")
print("- pycocoevalcap (BLEU, METEOR, ROUGE-L, CIDEr)")
print("\n✅ ƯU ĐIỂM pycocoevalcap:")
print("- Không cần Java (khác với nlg-eval)")
print("- Cài đặt đơn giản, ổn định hơn")
print("- Tính được đầy đủ metrics: BLEU-1,2,3,4, METEOR, ROUGE-L, CIDEr")


📦 Đang cài đặt pycocoevalcap cho evaluation metrics...
  Preparing metadata (setup.py) ... done

✓ Đã cài đặt xong!

📝 Các thư viện đã cài:
- NumPy < 1.24 (tương thích với np.long)
- tqdm, boto3, requests, pandas
- pycocoevalcap (BLEU, METEOR, ROUGE-L, CIDEr)

✅ ƯU ĐIỂM pycocoevalcap:
- Không cần Java (khác với nlg-eval)
- Cài đặt đơn giản, ổn định hơn
- Tính được đầy đủ metrics: BLEU-1,2,3,4, METEOR, ROUGE-L, CIDEr


---
## Bước 1: Chuẩn bị BERT model

### 1.1 Download BERT base model files

In [5]:
%%bash
# Tạo thư mục cho BERT
mkdir -p modules/bert-base-uncased
cd modules/bert-base-uncased

# Download vocab file
wget -q https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt
mv bert-base-uncased-vocab.txt vocab.txt

# Download BERT model
wget -q https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz
tar -xzf bert-base-uncased.tar.gz
rm bert-base-uncased.tar.gz

cd ../..
echo "✓ BERT model downloaded successfully!"
ls -lh modules/bert-base-uncased/

✓ BERT model downloaded successfully!
total 421M
-rw-r--r-- 1  501 staff  313 Oct 18  2018 bert_config.json
-rw-r--r-- 1  501 staff 421M Nov 14  2018 pytorch_model.bin
-rw-r--r-- 1 root root  227K Nov 30  2018 vocab.txt


---
## Bước 2: Tải Pretrained Weights và Dataset

### 2.1 Tải UniVL pretrained weights

In [ ]:
%%bash
# Tạo thư mục weight
mkdir -p weight

# Download pretrained weight
echo "Downloading UniVL pretrained weight (this may take a few minutes)..."
wget -q --show-progress https://github.com/microsoft/UniVL/releases/download/v0/univl.pretrained.bin -O weight/univl.pretrained.bin

echo "✓ UniVL pretrained weight downloaded!"
ls -lh weight/

### 2.2 Tải MSRVTT dataset

Dataset bao gồm:
- CSV files: Chứa thông tin về video IDs
- JSON file: Chứa captions và metadata  
- Pickle file: Chứa S3D video features (1024-dim, ~1.2GB)

In [6]:
%%bash
# Tạo thư mục data
mkdir -p data

# Download MSRVTT dataset
echo "Downloading MSRVTT dataset (this will take several minutes, ~1.2GB)..."
wget -q --show-progress https://github.com/microsoft/UniVL/releases/download/v0/msrvtt.zip -O data/msrvtt.zip

# Giải nén
echo "Extracting dataset..."
unzip -q data/msrvtt.zip -d data/

# Xóa file zip để tiết kiệm dung lượng
rm data/msrvtt.zip

echo "✓ MSRVTT dataset downloaded and extracted!"
ls -lh data/msrvtt/

Extracting dataset...
✓ MSRVTT dataset downloaded and extracted!
total 314M
-rw-r--r-- 1 root root  23M Dec 18  2020 MSRVTT_data.json
-rw-r--r-- 1 root root  73K Dec 18  2020 MSRVTT_JSFUSION_test.csv
-rw-r--r-- 1 root root  68K Dec 18  2020 MSRVTT_train.7k.csv
-rw-r--r-- 1 root root  87K Dec 18  2020 MSRVTT_train.9k.csv
-rw-r--r-- 1 root root 292M Jan  5  2021 msrvtt_videos_features.pickle
-rw-r--r-- 1 root root  246 Apr  8  2021 README.md



     0K .......... .......... .......... .......... ..........  0% 4.14M 60s
    50K .......... .......... .......... .......... ..........  0% 13.8M 39s
   100K .......... .......... .......... .......... ..........  0% 6.22M 39s
   150K .......... .......... .......... .......... ..........  0% 17.8M 33s
   200K .......... .......... .......... .......... ..........  0% 29.0M 28s
   250K .......... .......... .......... .......... ..........  0% 8.07M 28s
   300K .......... .......... .......... .......... ..........  0% 84.4M 25s
   350K .......... .......... .......... .......... ..........  0% 27.0M 23s
   400K .......... .......... .......... .......... ..........  0% 53.9M 21s
   450K .......... .......... .......... .......... ..........  0% 59.3M 19s
   500K .......... .......... .......... .......... ..........  0% 63.6M 18s
   550K .......... .......... .......... .......... ..........  0% 8.23M 19s
   600K .......... .......... .......... .......... ..........  0%  232M 17

---
## Bước 3: Khám phá Dataset MSRVTT

### 3.1 Xem cấu trúc CSV files

In [7]:
import os
import urllib.request
from pathlib import Path

# Tạo thư mục weight
weight_dir = Path("./weight")
weight_dir.mkdir(exist_ok=True)

pretrained_weight = weight_dir / "univl.pretrained.bin"

if not pretrained_weight.exists():
    print("Đang tải pretrained weight...")
    url = "https://github.com/microsoft/UniVL/releases/download/v0/univl.pretrained.bin"
    urllib.request.urlretrieve(url, pretrained_weight)
    print(f"✓ Đã tải xong: {pretrained_weight}")
else:
    print(f"✓ Pretrained weight đã tồn tại: {pretrained_weight}")

Đang tải pretrained weight...
✓ Đã tải xong: weight/univl.pretrained.bin


### 2.2 Tải BERT base model

Tải BERT tokenizer và config files:

In [9]:
import urllib.request
import tarfile

bert_dir = Path("./modules/bert-base-uncased")
bert_dir.mkdir(parents=True, exist_ok=True)

# Tải vocab file
vocab_file = bert_dir / "vocab.txt"
if not vocab_file.exists():
    print("Đang tải BERT vocab...")
    vocab_url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt"
    urllib.request.urlretrieve(vocab_url, vocab_file)
    print("✓ Đã tải vocab.txt")

# Tải BERT model files
bert_model = bert_dir / "pytorch_model.bin"
if not bert_model.exists():
    print("Đang tải BERT model...")
    bert_url = "https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz"
    tar_file = bert_dir / "bert-base-uncased.tar.gz"
    urllib.request.urlretrieve(bert_url, tar_file)
    
    # Giải nén
    with tarfile.open(tar_file, 'r:gz') as tar:
        tar.extractall(path=bert_dir)
    tar_file.unlink()  # Xóa file tar
    print("✓ Đã tải và giải nén BERT model")

print("\n✓ BERT files đã sẵn sàng")
print(f"  - vocab.txt: {vocab_file.exists()}")
print(f"  - pytorch_model.bin: {bert_model.exists()}")


✓ BERT files đã sẵn sàng
  - vocab.txt: True
  - pytorch_model.bin: True


### 2.3 Tải MSRVTT dataset

Dataset bao gồm:
- CSV files: Chứa thông tin về video IDs
- JSON file: Chứa captions và metadata
- Pickle file: Chứa S3D video features (1024-dim vectors)

In [10]:
import urllib.request
import zipfile

data_dir = Path("./data")
data_dir.mkdir(exist_ok=True)

msrvtt_dir = data_dir / "msrvtt"

if not msrvtt_dir.exists():
    print("Đang tải MSRVTT dataset...")
    zip_file = data_dir / "msrvtt.zip"
    url = "https://github.com/microsoft/UniVL/releases/download/v0/msrvtt.zip"
    
    urllib.request.urlretrieve(url, zip_file)
    print("Đang giải nén...")
    
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(data_dir)
    
    zip_file.unlink()
    print(f"✓ Dataset đã được giải nén vào: {msrvtt_dir}")
else:
    print(f"✓ MSRVTT dataset đã tồn tại: {msrvtt_dir}")

# Kiểm tra các file cần thiết
print("\nCác file trong dataset:")
for file in msrvtt_dir.glob("*"):
    print(f"  - {file.name} ({file.stat().st_size / 1024 / 1024:.2f} MB)")

✓ MSRVTT dataset đã tồn tại: data/msrvtt

Các file trong dataset:
  - msrvtt_videos_features.pickle (291.00 MB)
  - MSRVTT_data.json (22.04 MB)
  - MSRVTT_train.7k.csv (0.07 MB)
  - README.md (0.00 MB)
  - MSRVTT_JSFUSION_test.csv (0.07 MB)
  - MSRVTT_train.9k.csv (0.08 MB)


### 2.4 Khám phá cấu trúc dữ liệu MSRVTT

In [7]:
import json
import pandas as pd
import pickle

# Đọc train CSV
train_csv = pd.read_csv("data/msrvtt/MSRVTT_train.9k.csv")
print("Train CSV shape:", train_csv.shape)
print("\nTrain CSV columns:", train_csv.columns.tolist())
print("\nMẫu dữ liệu train CSV:")
print(train_csv.head())

# Đọc test CSV
test_csv = pd.read_csv("data/msrvtt/MSRVTT_JSFUSION_test.csv")
print("\n" + "="*50)
print("Test CSV shape:", test_csv.shape)
print("\nMẫu dữ liệu test CSV:")
print(test_csv.head())

Train CSV shape: (9000, 1)

Train CSV columns: ['video_id']

Mẫu dữ liệu train CSV:
  video_id
0   video0
1   video1
2   video2
3   video3
4   video4

Test CSV shape: (1000, 4)

Mẫu dữ liệu test CSV:
    key  vid_key   video_id                                    sentence
0  ret0  msr9770  video9770  a person is connecting something to system
1  ret1  msr9771  video9771               a little girl does gymnastics
2  ret2  msr7020  video7020  a woman creating a fondant baby and flower
3  ret3  msr9773  video9773              a boy plays grand theft auto 5
4  ret4  msr7026  video7026       a man is giving a review on a vehicle


In [12]:
# Đọc JSON data
with open("data/msrvtt/MSRVTT_data.json", 'r') as f:
    msrvtt_data = json.load(f)

print("Keys trong JSON file:", msrvtt_data.keys())
print(f"\nSố lượng videos: {len(msrvtt_data['videos'])}")
print(f"Số lượng sentences: {len(msrvtt_data['sentences'])}")

# Xem mẫu video info
print("\nMẫu video info:")
print(json.dumps(msrvtt_data['videos'][0], indent=2))

# Xem mẫu sentences
print("\nMẫu sentences cho video đầu tiên:")
for i, sent in enumerate(msrvtt_data['sentences'][:5]):
    if sent['video_id'] == msrvtt_data['videos'][0]['video_id']:
        print(f"  {i+1}. {sent['caption']}")

Keys trong JSON file: dict_keys(['info', 'videos', 'sentences'])

Số lượng videos: 10000
Số lượng sentences: 200000

Mẫu video info:
{
  "category": 9,
  "url": "https://www.youtube.com/watch?v=9lZi22qLlEo",
  "video_id": "video0",
  "start time": 137.72,
  "end time": 149.44,
  "split": "train",
  "id": 0
}

Mẫu sentences cho video đầu tiên:


In [8]:
# Đọc features pickle
with open("data/msrvtt/msrvtt_videos_features.pickle", 'rb') as f:
    video_features = pickle.load(f)

print(f"Số lượng videos có features: {len(video_features)}")

# Lấy một video bất kỳ để xem shape
sample_video_id = list(video_features.keys())[7010]
sample_features = video_features[sample_video_id]

print(f"\nMẫu video ID: {sample_video_id}")
print(f"Feature shape: {sample_features.shape}")
print(f"Feature type: {type(sample_features)}")
print(f"Feature dtype: {sample_features.dtype}")
print(f"\nGiải thích: Features có shape [num_frames, 1024]")
print(f"  - num_frames: Số lượng frames trong video (1 frame/giây)")
print(f"  - 1024: Dimension của S3D features")

Số lượng videos có features: 10000

Mẫu video ID: video2793
Feature shape: (11, 1024)
Feature type: <class 'numpy.ndarray'>
Feature dtype: float16

Giải thích: Features có shape [num_frames, 1024]
  - num_frames: Số lượng frames trong video (1 frame/giây)
  - 1024: Dimension của S3D features


---
## Bước 3: Hiểu cấu trúc model UniVL

### 3.1 Kiến trúc model

In [7]:
# Import các modules cần thiết
import sys
sys.path.append('./')

from modules.tokenization import BertTokenizer
from modules.modeling import UniVL
import torch

# Khởi tạo tokenizer
tokenizer = BertTokenizer.from_pretrained(
    "modules/bert-base-uncased", 
    do_lower_case=True
)

print(f"✓ Tokenizer vocab size: {len(tokenizer.vocab)}")
print(f"\nMẫu tokenization:")
sample_text = "A man is playing guitar on the stage"
tokens = tokenizer.tokenize(sample_text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"  Text: {sample_text}")
print(f"  Tokens: {tokens}")
print(f"  Token IDs: {token_ids}")

modules/bert-base-uncased/vocab.txt
✓ Tokenizer vocab size: 30522

Mẫu tokenization:
  Text: A man is playing guitar on the stage
  Tokens: ['a', 'man', 'is', 'playing', 'guitar', 'on', 'the', 'stage']
  Token IDs: [1037, 2158, 2003, 2652, 2858, 2006, 1996, 2754]


### 3.2 Các hyperparameters quan trọng

**Theo README và code, các tham số cho MSRVTT caption:**

| Tham số | Giá trị | Ý nghĩa |
|---------|---------|----------|
| `max_words` | 48 | Độ dài tối đa của caption (tokens) |
| `max_frames` | 48 | Số frames tối đa từ video |
| `batch_size` | 128 | Kích thước batch cho training |
| `batch_size_val` | 32 | Kích thước batch cho evaluation |
| `lr` | 3e-5 | Learning rate |
| `epochs` | 5 | Số epoch training |
| `visual_num_hidden_layers` | 6 | Số layers của visual encoder |
| `decoder_num_hidden_layers` | 3 | Số layers của decoder |
| `stage_two` | True | Sử dụng decoder (Stage II) |

**Kết quả kỳ vọng:**
```
BLEU_1: 0.8051, BLEU_2: 0.6672, BLEU_3: 0.5342, BLEU_4: 0.4179
METEOR: 0.2894, ROUGE_L: 0.6078, CIDEr: 0.5004
```

---
## Bước 4: Thiết lập training script

### 4.1 Tạo output directory

In [16]:
output_dir = Path("./ckpts/ckpt_msrvtt_caption")
output_dir.mkdir(parents=True, exist_ok=True)
print(f"✓ Output directory: {output_dir}")

✓ Output directory: ckpts/ckpt_msrvtt_caption


### 4.2 Kiểm tra cấu hình GPU và distributed training

In [17]:
import torch

n_gpu = torch.cuda.device_count()
print(f"Số lượng GPU khả dụng: {n_gpu}")

if n_gpu > 0:
    for i in range(n_gpu):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        
    # Khuyến nghị
    if n_gpu >= 4:
        print("\n✓ Có thể sử dụng 4 GPUs như trong README")
        print("  Lệnh: python -m torch.distributed.launch --nproc_per_node=4 main_task_caption.py ...")
    elif n_gpu > 1:
        print(f"\n⚠ Có {n_gpu} GPUs. Có thể điều chỉnh --nproc_per_node={n_gpu}")
    else:
        print("\n⚠ Chỉ có 1 GPU. Có thể train nhưng sẽ chậm hơn.")
        print("  Cân nhắc giảm batch_size nếu gặp OOM error")
else:
    print("\n✗ Không có GPU. CPU training sẽ rất chậm.")

Số lượng GPU khả dụng: 2
  GPU 0: Tesla T4
    Memory: 14.74 GB
  GPU 1: Tesla T4
    Memory: 14.74 GB

⚠ Có 2 GPUs. Có thể điều chỉnh --nproc_per_node=2


---
## Bước 5: Training Command

### 5.1 Command cho Multi-GPU Training (Khuyến nghị)

In [18]:
# Tạo training script cho Kaggle (single GPU hoặc multi-GPU)
import torch

n_gpu = torch.cuda.device_count()
print(f"Số GPU khả dụng: {n_gpu}")

# Điều chỉnh batch size dựa trên số GPU
if n_gpu == 0:
    print("❌ ERROR: Không có GPU! Hãy bật GPU trong Kaggle Settings.")
elif n_gpu == 1:
    nproc = 1
    batch_size = 32
    batch_size_val = 16
    grad_accum = 4
    print(f"✓ Sử dụng 1 GPU, batch_size={batch_size}, gradient_accumulation_steps={grad_accum}")
elif n_gpu == 2:
    nproc = 2
    batch_size = 64
    batch_size_val = 32
    grad_accum = 2
    print(f"✓ Sử dụng 2 GPUs, batch_size={batch_size}, gradient_accumulation_steps={grad_accum}")
else:
    nproc = n_gpu
    batch_size = 128
    batch_size_val = 32
    grad_accum = 1
    print(f"✓ Sử dụng {n_gpu} GPUs, batch_size={batch_size}")

# Tạo training command - Sử dụng torchrun thay vì torch.distributed.launch
training_command = f"""torchrun --nproc_per_node={nproc} --standalone \\
main_task_caption.py \\
--do_train --num_thread_reader=4 \\
--epochs=5 --batch_size={batch_size} \\
--n_display=50 \\
--train_csv data/msrvtt/MSRVTT_train.9k.csv \\
--val_csv data/msrvtt/MSRVTT_JSFUSION_test.csv \\
--data_path data/msrvtt/MSRVTT_data.json \\
--features_path data/msrvtt/msrvtt_videos_features.pickle \\
--output_dir ckpts/ckpt_msrvtt_caption --bert_model bert-base-uncased \\
--do_lower_case --lr 3e-5 --max_words 48 --max_frames 48 \\
--batch_size_val {batch_size_val} --visual_num_hidden_layers 6 \\
--decoder_num_hidden_layers 3 --datatype msrvtt --stage_two \\
--init_model weight/univl.pretrained.bin \\
--gradient_accumulation_steps={grad_accum}
"""

print("\n" + "="*80)
print("TRAINING COMMAND (Updated for PyTorch 2.x):")
print("="*80)
print(training_command)
print("="*80)

# Lưu vào file
with open("train_msrvtt_caption.sh", 'w') as f:
    f.write(training_command)
print("\n✓ Command đã được lưu vào: train_msrvtt_caption.sh")
print("\n📝 LƯU Ý: Đã chuyển từ 'torch.distributed.launch' sang 'torchrun'")
print("   để tương thích với PyTorch 2.x trên Kaggle")

Số GPU khả dụng: 2
✓ Sử dụng 2 GPUs, batch_size=64, gradient_accumulation_steps=2

TRAINING COMMAND (Updated for PyTorch 2.x):
torchrun --nproc_per_node=2 --standalone \
main_task_caption.py \
--do_train --num_thread_reader=4 \
--epochs=5 --batch_size=64 \
--n_display=50 \
--train_csv data/msrvtt/MSRVTT_train.9k.csv \
--val_csv data/msrvtt/MSRVTT_JSFUSION_test.csv \
--data_path data/msrvtt/MSRVTT_data.json \
--features_path data/msrvtt/msrvtt_videos_features.pickle \
--output_dir ckpts/ckpt_msrvtt_caption --bert_model bert-base-uncased \
--do_lower_case --lr 3e-5 --max_words 48 --max_frames 48 \
--batch_size_val 32 --visual_num_hidden_layers 6 \
--decoder_num_hidden_layers 3 --datatype msrvtt --stage_two \
--init_model weight/univl.pretrained.bin \
--gradient_accumulation_steps=2


✓ Command đã được lưu vào: train_msrvtt_caption.sh

📝 LƯU Ý: Đã chuyển từ 'torch.distributed.launch' sang 'torchrun'
   để tương thích với PyTorch 2.x trên Kaggle


---
## Bước 6: Chạy Training

### 6.1 Khởi tạo output directory

In [19]:
# Tạo thư mục output
!mkdir -p ckpts/ckpt_msrvtt_caption
print("✓ Output directory created!")

✓ Output directory created!


### 6.2 Bắt đầu training

**Lưu ý quan trọng:**
- Training sẽ mất khoảng **3-6 giờ** tùy theo GPU
- Kaggle có giới hạn **9 giờ** GPU mỗi session
- Model sẽ được lưu checkpoint sau mỗi epoch
- Có thể dừng và resume nếu cần

In [ ]:
%%time
# Chạy training script
!bash train_msrvtt_caption.sh